In [1]:
import torch.nn as nn
import torch
import zenkai

import torch.utils.data as torch_data



In [4]:
# torch.manual_seed(1)
# base = nn.Linear(2, 2)
# wrapped = wrap.CriterionNNWrap(
#     base, zenkai.ThLoss('MSELoss'), to_step_x=False
# )

# x = torch.randn(4, 2)
# y = wrapped(x)

# print(y.grad_fn)
# (torch.rand(4, 2) - y).pow(2).mean().backward()

tensor([[-0.1275,  0.1329],
        [-0.0837,  0.0726],
        [-0.0837,  0.0329],
        [-0.0947,  0.0697]])


In [2]:
torch.manual_seed(1)

class Net(nn.Module):

    def __init__(self, in_features, mid_features, out_features):
        super().__init__()

        self.layer1 = zenkai.GradLearner(
            nn.Linear(in_features, mid_features), 
            zenkai.OptimFactory('Adam', lr=1e-3).comp(),
            zenkai.ThLoss('MSELoss', reduction='sum')
        )

        self.layer2 = zenkai.GradLearner(
            nn.Linear(mid_features, out_features), 
            zenkai.OptimFactory('Adam', lr=1e-3).comp(),
            zenkai.ThLoss('MSELoss', reduction='sum')
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:

        y = self.layer1(x)
        return self.layer2(y)


x = torch.randn(1000, 4)
base = Net(4, 8, 6)
t = base(x).detach()
trainable = Net(4, 8, 6)
zenkai.set_net_lmode(
    trainable, zenkai.LMode.WithStep
)
dataset = torch_data.TensorDataset(x, t)

for i in range(100):
    for x_i, t_i in torch_data.DataLoader(dataset, shuffle=True, batch_size=128):

        y_i = trainable(x_i)
        loss = (y_i - t_i).pow(2).mean()
        loss.backward()
        print(loss.item())


0.35644686222076416
0.3233437240123749
0.33525657653808594
0.341363787651062
0.2968562841415405
0.32389336824417114
0.3686041533946991
0.30159878730773926
0.3302888870239258
0.3487548530101776
0.3145020604133606
0.2719280421733856
0.28386616706848145
0.28620609641075134
0.2667539417743683
0.31595900654792786
0.3175870180130005
0.2604183554649353
0.273172527551651
0.2769261598587036
0.27959081530570984
0.25832676887512207
0.27371302247047424
0.2561168074607849
0.27340927720069885
0.2733529806137085
0.2487257868051529
0.24388368427753448
0.2752922475337982
0.24453645944595337
0.24159367382526398
0.20021846890449524
0.22529317438602448
0.2025107592344284
0.23941971361637115
0.2439853399991989
0.21131867170333862
0.2448796182870865
0.2496546506881714
0.2186405211687088
0.2363387942314148
0.20325638353824615
0.21280087530612946
0.21780438721179962
0.22008632123470306
0.20476996898651123
0.19367139041423798
0.19839946925640106
0.18469493091106415
0.22030842304229736
0.22946691513061523
0.182